In [3]:
# Imports
import os
import sys
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

from matplotlib import pyplot as plt
import seaborn as sns

sns.set_style('dark')

In [585]:
# Get the username from terminal
username = sys.argv[1]

os.environ['SPOTIPY_CLIENT_ID'] = '509f1c0827f94cd7af9c842beb26e7ef'
os.environ['SPOTIPY_CLIENT_SECRET'] = '83fe375561354e6485bde0e952dc5d70'
os.environ['SPOTIPY_REDIRECT_URI'] = 'http://localhost:8888/callback/'

cid = '509f1c0827f94cd7af9c842beb26e7ef' # Client ID
secret = '83fe375561354e6485bde0e952dc5d70'   # Client Secret ID
username = '21ugtw76ynask7ohvvz626ndq' # Spotify Username

scope = 'user-library-read playlist-modify-public playlist-read-private'
redirect_uri = 'http://localhost:8888/'
#http://localhost:8888/callback/

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
token = util.prompt_for_user_token(username, scope, cid, secret, redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [310]:
genres = ['house', 'techno', 'dnb', 'hardstyle', 'trap']

df = pd.read_csv(r'C:\Users\Chetan\Desktop\College\Python\EDM Sub-genre classifier\playlists.csv')
df.head()

,username,playlistid,genre,title,url
0,Élite,5sPzNVd9MbUwmWd2mA90qB,house,Boiler Room | Techno House,https://open.spotify.com/playlist/5sPzNVd9MbUw...
1,Beatport,6Pbk82RPOKWD7k6dNcqq9f,house,Beatport Best New Tech House,https://open.spotify.com/playlist/6Pbk82RPOKWD...
2,Mac Peterson,7AUqarp6W3pf0tVFrRvzZw,house,Dirtybird Players,https://open.spotify.com/playlist/7AUqarp6W3pf...
3,Alex Meyer,1NQK1mGamaJmmwNE4iv9PT,house,House/Dirtybird,https://open.spotify.com/playlist/1NQK1mGamaJm...
4,Dave Lloyd,31QIVbCj6M2KkVxMepKS7b,house,Dirtybird//Techhouse by Mack,https://open.spotify.com/playlist/31QIVbCj6M2K...


In [6]:
# FIRST ATTEMPT AT MAKING A FUNCTION TO RETRIEVE FEATURES OF SONGS
# ISSUES FACED - UNABLE TO READ MORE THAN 100 SONGS IN A PLAYLIST

def get_features(username, playlist_id, genre):
    results = sp.user_playlist_tracks(username, playlist_id)
    songs = results['items']
    
    while results['next']:
        results = sp.next(results)
        songs.extend(results['items'])
    
    ids = []
        
    for i in range(len(songs)):
        ids.append(songs[i]['track']['id'])
    
    ids = list(filter(None, ids))
    
    features = []
    
    for i in range(0,len(ids),50):
        audio_features = sp.audio_features(ids[i:i+50])
        
        for track in audio_features:
            features.append(track)

    features = list(filter(None, features))
    df = pd.DataFrame(features)
    df['genre'] = genre
    return df

In [7]:
def create_df(PlayLists):
    result = pd.DataFrame()
    
    # Extract songs from spotify API and append to dataframe
    for i in range(len(PlayLists)):
        df = get_features(PlayLists['username'][i], PlayLists['playlistid'][i], PlayLists['genre'][i])
        df['title'] = PlayLists['title'][i]
        result = pd.concat([result, df], ignore_index=True)
    
    return result

In [8]:
def analyze_playlist(creator, playlist_id):
    
    # Create empty dataframelimit=2000
    playlist_features_list = ["artist","album","track_name",  "track_id","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature"]
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    # Loop through every track in the playlist, extract features and append the features to the playlist df
    
    playlist = sp.user_playlist_tracks(creator, playlist_id)["items"]
    for track in playlist:
        # Create empty dict
        playlist_features = {}
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_features[feature] = audio_features[feature]
        
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
        
    return playlist_df

In [9]:
# These Functions can read more than 100 songs a playlist

def show_tracks(results, uriArray):
    for i, item in enumerate(results['items']):
        track = item['track']
        uriArray.append(track['id'])

def get_playlist_track_id(username, playlist_id):
    trackID = []
    results = sp.user_playlist(username, playlist_id)
    tracks = results['tracks']
    show_tracks(tracks, trackID)
    while tracks['next']:
            tracks = sp.next(tracks)
            show_tracks(tracks, trackID)
    return trackID

In [11]:
songs_all = []
for i in range (0, 93): #Put Length of the playlist wala list here
    songs = get_playlist_track_id(df['username'][i], df['playlistid'][i])
    songs_all.extend(songs)

In [12]:
len(songs_all)  # Correct list length for all playlist indexed 0,1,2

35996

In [13]:
sp.audio_features(tracks = '0m69QNuZVZDXhGHBGK1mQA')

[{'danceability': 0.74,
  'energy': 0.934,
  'key': 7,
  'loudness': -3.255,
  'mode': 1,
  'speechiness': 0.316,
  'acousticness': 0.00822,
  'instrumentalness': 0.000377,
  'liveness': 0.116,
  'valence': 0.685,
  'tempo': 124.936,
  'type': 'audio_features',
  'id': '0m69QNuZVZDXhGHBGK1mQA',
  'uri': 'spotify:track:0m69QNuZVZDXhGHBGK1mQA',
  'track_href': 'https://api.spotify.com/v1/tracks/0m69QNuZVZDXhGHBGK1mQA',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0m69QNuZVZDXhGHBGK1mQA',
  'duration_ms': 154680,
  'time_signature': 4}]

In [14]:
# THIS FUNCTION IS THE ORIGINAL WORKING FUNCTION FOR CORRECT DATAFRAME
# ISSUE FACED - TIMEOUT HO JATA HAI FOR SUCH A LARGE NUMBER OF SONGS


def song_feature_ext(playlist):
    result = pd.DataFrame()
    for i in playlist:
        song_feat = sp.audio_features(tracks = i)
        result = pd.concat([result, pd.Series(song_feat[0])], ignore_index=True, axis = 1)

    return result.T

In [16]:
# WITH THIS FUNCTION, INSTEAD OF GIVING ALL 35992 SONGS TO THE LOOP AT ONCE,
# WE GIVE IT IN SEGMENTS OF 3000-3000. THE TIME FOR THE RUNNING OF THE LOOP
# IS AROUND 9 MINUTES 
# 500 songs takes around 1:50 minutes

def song_feature_ext_initial(playlist):
    result = pd.DataFrame()
    for i in range(0, 500):
        song_feat = sp.audio_features(tracks = playlist[i])
        result = pd.concat([result, pd.Series(song_feat[0])], ignore_index=True, axis = 1)

    return result.T

In [39]:
songs = song_feature_ext_initial(songs_all)

In [40]:
songs

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.786,0.87,2,-5.839,1,0.0507,0.0493,0.215,0.0863,0.676,126.011,audio_features,7LVEBvmKB5uDTg0CGqDo3p,spotify:track:7LVEBvmKB5uDTg0CGqDo3p,https://api.spotify.com/v1/tracks/7LVEBvmKB5uD...,https://api.spotify.com/v1/audio-analysis/7LVE...,192482,4
1,0.74,0.934,7,-3.255,1,0.316,0.00822,0.000377,0.116,0.685,124.936,audio_features,0m69QNuZVZDXhGHBGK1mQA,spotify:track:0m69QNuZVZDXhGHBGK1mQA,https://api.spotify.com/v1/tracks/0m69QNuZVZDX...,https://api.spotify.com/v1/audio-analysis/0m69...,154680,4
2,0.889,0.859,11,-5.11,1,0.205,0.0216,0.106,0.737,0.828,124.958,audio_features,4PfzZIFxK10aHXupOm4qEu,spotify:track:4PfzZIFxK10aHXupOm4qEu,https://api.spotify.com/v1/tracks/4PfzZIFxK10a...,https://api.spotify.com/v1/audio-analysis/4Pfz...,142560,4
3,0.84,0.868,11,-6.173,0,0.0721,0.0259,0.573,0.0891,0.851,126.001,audio_features,6uABa7vKOgFqXnwvk8NBr5,spotify:track:6uABa7vKOgFqXnwvk8NBr5,https://api.spotify.com/v1/tracks/6uABa7vKOgFq...,https://api.spotify.com/v1/audio-analysis/6uAB...,203810,4
4,0.74,0.989,10,-4.193,1,0.0469,0.00276,0.956,0.306,0.934,125.001,audio_features,6fcHhf9bix2uLbKHApSkNy,spotify:track:6fcHhf9bix2uLbKHApSkNy,https://api.spotify.com/v1/tracks/6fcHhf9bix2u...,https://api.spotify.com/v1/audio-analysis/6fcH...,142080,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.734,0.977,0,-6.196,1,0.069,0.0187,0.109,0.0986,0.704,123.972,audio_features,1HOFLt87VuclSyIIVNMjpy,spotify:track:1HOFLt87VuclSyIIVNMjpy,https://api.spotify.com/v1/tracks/1HOFLt87Vucl...,https://api.spotify.com/v1/audio-analysis/1HOF...,185000,4
496,0.707,0.973,3,-2.47,0,0.0508,0.0144,0.897,0.0401,0.585,125.001,audio_features,5JH18IpSAaCSWljNKalBzQ,spotify:track:5JH18IpSAaCSWljNKalBzQ,https://api.spotify.com/v1/tracks/5JH18IpSAaCS...,https://api.spotify.com/v1/audio-analysis/5JH1...,255360,4
497,0.795,0.981,6,-4.665,1,0.0517,0.00119,0.91,0.0519,0.666,128,audio_features,6It7eAXKK59k0skPxWNwP1,spotify:track:6It7eAXKK59k0skPxWNwP1,https://api.spotify.com/v1/tracks/6It7eAXKK59k...,https://api.spotify.com/v1/audio-analysis/6It7...,196406,4
498,0.795,0.797,10,-5.96,0,0.0634,0.00438,0.302,0.129,0.627,124.952,audio_features,2M72ge9OrR7muKhcBi509v,spotify:track:2M72ge9OrR7muKhcBi509v,https://api.spotify.com/v1/tracks/2M72ge9OrR7m...,https://api.spotify.com/v1/audio-analysis/2M72...,189614,4


In [618]:
def song_feature_ext_subsequent(playlist):
    result = pd.DataFrame()
    for i in range(30100, 30200):
        song_feat = sp.audio_features(tracks = playlist[i])
        result = pd.concat([result, pd.Series(song_feat[0])], ignore_index=True, axis = 1)

    return result.T

In [619]:
songs_subsequent = song_feature_ext_subsequent(songs_all)

TypeError: 'NoneType' object is not iterable

In [580]:
songs_subsequent

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.71,0.896,8,-2.471,0,0.319,0.119,0.00555,0.0556,0.223,72.4,audio_features,6bjfVRftOuvWPlnE5MzCv4,spotify:track:6bjfVRftOuvWPlnE5MzCv4,https://api.spotify.com/v1/tracks/6bjfVRftOuvW...,https://api.spotify.com/v1/audio-analysis/6bjf...,205241,4
1,0.644,0.872,11,-1.167,1,0.148,0.000571,0.004,0.116,0.346,76.031,audio_features,2C7KSL0jA5kAh2xB9nPIY7,spotify:track:2C7KSL0jA5kAh2xB9nPIY7,https://api.spotify.com/v1/tracks/2C7KSL0jA5kA...,https://api.spotify.com/v1/audio-analysis/2C7K...,198404,4
2,0.596,0.938,11,-2.155,1,0.131,0.00407,0.00697,0.132,0.552,75.029,audio_features,61ETloLtIoQApPPdBaA7PU,spotify:track:61ETloLtIoQApPPdBaA7PU,https://api.spotify.com/v1/tracks/61ETloLtIoQA...,https://api.spotify.com/v1/audio-analysis/61ET...,188800,4
3,0.453,0.756,1,-3.06,0,0.141,0.000124,0.196,0.288,0.29,139.617,audio_features,31BFq5U0rgZNAxSu0KeTBg,spotify:track:31BFq5U0rgZNAxSu0KeTBg,https://api.spotify.com/v1/tracks/31BFq5U0rgZN...,https://api.spotify.com/v1/audio-analysis/31BF...,221143,4
4,0.654,0.89,9,-2.562,1,0.215,0.00726,9.96e-05,0.665,0.129,144.974,audio_features,530xeorCOsbwQsUVVearth,spotify:track:530xeorCOsbwQsUVVearth,https://api.spotify.com/v1/tracks/530xeorCOsbw...,https://api.spotify.com/v1/audio-analysis/530x...,264828,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,0.538,0.789,4,-6.05,0,0.229,0.015,1.28e-06,0.297,0.152,147.647,audio_features,0L4oYjLVlG2I1oBPNKdOqy,spotify:track:0L4oYjLVlG2I1oBPNKdOqy,https://api.spotify.com/v1/tracks/0L4oYjLVlG2I...,https://api.spotify.com/v1/audio-analysis/0L4o...,178155,4
492,0.665,0.459,8,-9.867,0,0.161,0.212,0,0.0919,0.525,137.633,audio_features,3ey7v4OxmWKgp8ynQnk4XW,spotify:track:3ey7v4OxmWKgp8ynQnk4XW,https://api.spotify.com/v1/tracks/3ey7v4OxmWKg...,https://api.spotify.com/v1/audio-analysis/3ey7...,184760,4
493,0.627,0.771,2,-6.57,1,0.0562,0.000278,0.472,0.107,0.208,105.195,audio_features,4UL2eutSPzASZERkGtyMKZ,spotify:track:4UL2eutSPzASZERkGtyMKZ,https://api.spotify.com/v1/tracks/4UL2eutSPzAS...,https://api.spotify.com/v1/audio-analysis/4UL2...,144036,4
494,0.474,0.708,1,-4.494,1,0.168,0.0496,4.43e-05,0.0986,0.328,149.857,audio_features,72XViAl7vmQCa8liuVZjtg,spotify:track:72XViAl7vmQCa8liuVZjtg,https://api.spotify.com/v1/tracks/72XViAl7vmQC...,https://api.spotify.com/v1/audio-analysis/72XV...,156800,4


In [581]:
songs = pd.concat([songs, songs_subsequent]).reset_index(drop = True)

In [582]:
songs

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.786,0.87,2,-5.839,1,0.0507,0.0493,0.215,0.0863,0.676,126.011,audio_features,7LVEBvmKB5uDTg0CGqDo3p,spotify:track:7LVEBvmKB5uDTg0CGqDo3p,https://api.spotify.com/v1/tracks/7LVEBvmKB5uD...,https://api.spotify.com/v1/audio-analysis/7LVE...,192482,4
1,0.74,0.934,7,-3.255,1,0.316,0.00822,0.000377,0.116,0.685,124.936,audio_features,0m69QNuZVZDXhGHBGK1mQA,spotify:track:0m69QNuZVZDXhGHBGK1mQA,https://api.spotify.com/v1/tracks/0m69QNuZVZDX...,https://api.spotify.com/v1/audio-analysis/0m69...,154680,4
2,0.889,0.859,11,-5.11,1,0.205,0.0216,0.106,0.737,0.828,124.958,audio_features,4PfzZIFxK10aHXupOm4qEu,spotify:track:4PfzZIFxK10aHXupOm4qEu,https://api.spotify.com/v1/tracks/4PfzZIFxK10a...,https://api.spotify.com/v1/audio-analysis/4Pfz...,142560,4
3,0.84,0.868,11,-6.173,0,0.0721,0.0259,0.573,0.0891,0.851,126.001,audio_features,6uABa7vKOgFqXnwvk8NBr5,spotify:track:6uABa7vKOgFqXnwvk8NBr5,https://api.spotify.com/v1/tracks/6uABa7vKOgFq...,https://api.spotify.com/v1/audio-analysis/6uAB...,203810,4
4,0.74,0.989,10,-4.193,1,0.0469,0.00276,0.956,0.306,0.934,125.001,audio_features,6fcHhf9bix2uLbKHApSkNy,spotify:track:6fcHhf9bix2uLbKHApSkNy,https://api.spotify.com/v1/tracks/6fcHhf9bix2u...,https://api.spotify.com/v1/audio-analysis/6fcH...,142080,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34491,0.538,0.789,4,-6.05,0,0.229,0.015,1.28e-06,0.297,0.152,147.647,audio_features,0L4oYjLVlG2I1oBPNKdOqy,spotify:track:0L4oYjLVlG2I1oBPNKdOqy,https://api.spotify.com/v1/tracks/0L4oYjLVlG2I...,https://api.spotify.com/v1/audio-analysis/0L4o...,178155,4
34492,0.665,0.459,8,-9.867,0,0.161,0.212,0,0.0919,0.525,137.633,audio_features,3ey7v4OxmWKgp8ynQnk4XW,spotify:track:3ey7v4OxmWKgp8ynQnk4XW,https://api.spotify.com/v1/tracks/3ey7v4OxmWKg...,https://api.spotify.com/v1/audio-analysis/3ey7...,184760,4
34493,0.627,0.771,2,-6.57,1,0.0562,0.000278,0.472,0.107,0.208,105.195,audio_features,4UL2eutSPzASZERkGtyMKZ,spotify:track:4UL2eutSPzASZERkGtyMKZ,https://api.spotify.com/v1/tracks/4UL2eutSPzAS...,https://api.spotify.com/v1/audio-analysis/4UL2...,144036,4
34494,0.474,0.708,1,-4.494,1,0.168,0.0496,4.43e-05,0.0986,0.328,149.857,audio_features,72XViAl7vmQCa8liuVZjtg,spotify:track:72XViAl7vmQCa8liuVZjtg,https://api.spotify.com/v1/tracks/72XViAl7vmQC...,https://api.spotify.com/v1/audio-analysis/72XV...,156800,4


In [622]:
songs.to_csv(r'C:\Users\Chetan\Desktop\College\Python\EDM Sub-genre classifier\songs.csv', index = False)

In [ ]:
result = pd.DataFrame()

for i in range(len(songs_all)):
    df = sp.audio_features(songs_all[i])
    result = pd.concat([result, df], ignore_index=True)

result = result.drop_duplicates()
return result

In [ ]:
for i in range(0, len(songs_all)):
    pd.DataFrame(sp.audio_features(songs_all[i]))


In [ ]:
songs_sahi.head(10)

def get_playlist_track_id1(username, playlist_id, genre):
    trackID = []
    results = sp.user_playlist(username, playlist_id)
    tracks = results['tracks']
    show_tracks(tracks, trackID)
    while tracks['next']:
            tracks = sp.next(tracks)
            show_tracks(tracks, trackID)
    return trackID, genre

In [ ]:
sp.audio_features(tracks=['6uABa7vKOgFqXnwvk8NBr5'])

In [ ]:
sp.audio_features(tracks = get_playlist_track_id(df['username'][0], df['playlistid'][0]))

In [ ]:
songs_all

In [ ]:
song_feat1 = sp.audio_features(tracks = '6uABa7vKOgFqXnwvk8NBr5')